变量管理

本文档介绍如何通过tf.get_variable 和 tf.variable_scope来管理变量

In [3]:
import tensorflow as tf

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


在名字为foo的命名空间中创建名字为v的变量

In [4]:
with tf.variable_scope("foo"):
    v = tf.get_variable(
        "v", [1], initializer=tf.constant_initializer(1.0)
    )

因为在命名空间已经存在名字为v的变量，所以下面的代码将会报错

In [5]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "E:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1717, in __init__
    self._traceback = tf_stack.extract_stack()
  File "E:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3155, in create_op
    op_def=op_def)
  File "E:\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 454, in new_func
    return func(*args, **kwargs)


在生成上下文管理器时，将参数resue设置为True。这样tf.get_variable函数将直接获取已经声明的变量

In [6]:
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
    print(v==v1)

True


将参数reuse设置为True时，tf.variable_scope将只能获取已经创建过的变量。因为在命名空间bar中还没有创建变量v，所以下面的代码将会报错

In [7]:
with tf.variable_scope("bar", reuse=True):
    v = tf.get_variable("v", [1])

ValueError: Variable bar/v does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?

tf.variable_scope函数生成的上下文管理器也会创建一个TensorFlow中的命名空间，在命名空间中创建的变量名称都会带上这个命名空间名作为前缀。所以tf.variable_scope函数除了可以控制tf.get_variable执行的功能之外，这个函数也提供了一个管理变量命名空间的方式。

In [10]:
v1 = tf.get_variable("v1", [1])
print(v1.name)

v1:0


In [11]:
with tf.variable_scope("foo"):
    v2 = tf.get_variable("v2", [1])
    print(v2.name)

foo/v2:0


In [13]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v3 = tf.get_variable("v3",[1])
        print(v3.name)
    
    v4 = tf.get_variable("v4", [1])
    print(v4.name)

ValueError: Variable foo/bar/v3 already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "E:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1717, in __init__
    self._traceback = tf_stack.extract_stack()
  File "E:\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3155, in create_op
    op_def=op_def)
  File "E:\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 454, in new_func
    return func(*args, **kwargs)


创建一个名称为空的命名空间，并设置reuse=True

In [14]:
with tf.variable_scope("", reuse=True):
    v5=tf.get_variable("foo/bar/v3",[1])
    print(v5==v3)
    
    v6 = tf.get_variable("foo/v4",[1])
    print(v6 == v4)

True


ValueError: Variable foo/v4 does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?